# Quantum annealing time evolution
## Albert Molina

Here I will be coding my Hamiltonians and time evolution

In [4]:
import scipy as sp
import numpy as np
from scipy.sparse.linalg import eigsh, expm_multiply
import matplotlib.pyplot as plt